In [1]:
import pandas as pd

# Commit sharings

In [38]:
df = pd.read_csv('../data/interim/commit_sharings_df.csv')
#df

In [5]:
df.describe()

,NumberOfChatgptSharings,TotalNumberOfPrompts,TotalTokensOfPrompts,TotalTokensOfAnswers,NumberOfConversations,ModelGPT4,ModelGPT3.5,ModelOther,Status404
count,694.0,694.000000,694.000000,694.000000,694.000000,694.000000,694.000000,694.000000,694.000000
mean,1.0,2.769452,843.631124,960.103746,2.308357,0.873199,0.072046,0.051873,0.002882
std,0.0,4.822254,907.174052,1678.751116,4.650251,0.332990,0.258751,0.221931,0.053644
min,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.0,1.000000,514.250000,328.000000,1.000000,1.000000,0.000000,0.000000,0.000000
50%,1.0,1.000000,705.000000,523.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,1.0,2.000000,944.000000,843.250000,1.000000,1.000000,0.000000,0.000000,0.000000
max,1.0,64.000000,14690.000000,21311.000000,63.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
df.columns

Index(['Type', 'URL', 'Author', 'RepoName', 'RepoLanguage', 'Sha', 'Message',
       'AuthorAt', 'CommitAt', 'NumberOfChatgptSharings',
       'TotalNumberOfPrompts', 'TotalTokensOfPrompts', 'TotalTokensOfAnswers',
       'NumberOfConversations', 'ModelGPT4', 'ModelGPT3.5', 'ModelOther',
       'Status404'],
      dtype='object')

## Grouping by RepoName

In [13]:
grouped = df.groupby(by=['RepoName'], dropna=False)
repos_list = list(grouped.groups.keys())
len(repos_list)

76

In [39]:
#for name, group in grouped:
#    print(name)
#    print(group[['RepoName','RepoLanguage','Sha']])

In [29]:
grouped.size().describe()

count     76.000000
mean       9.131579
std       65.818405
min        1.000000
25%        1.000000
50%        1.000000
75%        1.000000
max      575.000000
dtype: float64

In [30]:
grouped_commits = grouped[['Sha']]
grouped_commits.count().describe()

,Sha
count,76.000000
mean,9.131579
std,65.818405
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,575.000000


In [16]:
grouped_counts = grouped[['NumberOfChatgptSharings',
       'TotalNumberOfPrompts', 'TotalTokensOfPrompts', 'TotalTokensOfAnswers',
       'NumberOfConversations', 'ModelGPT4', 'ModelGPT3.5', 'ModelOther',
       'Status404']]
grouped_counts.sum()

,NumberOfChatgptSharings,TotalNumberOfPrompts,TotalTokensOfPrompts,TotalTokensOfAnswers,NumberOfConversations,ModelGPT4,ModelGPT3.5,ModelOther,Status404
RepoName,,,,,,,,,
4lejandrito/fetchbook,1,5,328,2543,4,0,1,0,0
Atulaya123/reddit-next13-prisma-sql,1,2,189,209,1,0,1,0,0
Buildstarted/linksfordevs,1,64,516,7473,63,0,1,0,0
CPSC-1520/CPSC-1520-Workbook-A02-dgilleland,1,1,267,605,1,0,0,1,0
Cissucalris9/cuemaker2,1,17,5299,7520,16,0,1,0,0
...,...,...,...,...,...,...,...,...,...
veevyo/coding-exercises,1,1,10,533,1,0,1,0,0
waylon620/GPT-tutor,1,3,69,923,2,0,1,0,0
winksaville/scripts,1,27,1441,7331,26,1,0,0,0


## Final aggregation: `df_repo`

In [32]:
df_repo = grouped.agg({
        'RepoLanguage': 'first',
        'Sha': 'count',
        **{
            col: 'sum'
            for col in [
                'NumberOfChatgptSharings', 'Status404',
                'ModelGPT4', 'ModelGPT3.5', 'ModelOther',
                'TotalNumberOfPrompts', 'TotalTokensOfPrompts', 'TotalTokensOfAnswers',
                'NumberOfConversations', 
            ]
        }
    })
df_repo

,RepoLanguage,Sha,NumberOfChatgptSharings,Status404,ModelGPT4,ModelGPT3.5,ModelOther,TotalNumberOfPrompts,TotalTokensOfPrompts,TotalTokensOfAnswers,NumberOfConversations
RepoName,,,,,,,,,,,
4lejandrito/fetchbook,JavaScript,1,1,0,0,1,0,5,328,2543,4
Atulaya123/reddit-next13-prisma-sql,CSS,1,1,0,0,1,0,2,189,209,1
Buildstarted/linksfordevs,CSS,1,1,0,0,1,0,64,516,7473,63
CPSC-1520/CPSC-1520-Workbook-A02-dgilleland,CSS,1,1,0,0,0,1,1,267,605,1
Cissucalris9/cuemaker2,Python,1,1,0,0,1,0,17,5299,7520,16
...,...,...,...,...,...,...,...,...,...,...,...
veevyo/coding-exercises,C,1,1,0,0,1,0,1,10,533,1
waylon620/GPT-tutor,Batchfile,1,1,0,0,1,0,3,69,923,2
winksaville/scripts,Python,1,1,0,1,0,0,27,1441,7331,26


In [37]:
df_repo.drop(columns=['RepoLanguage']).sum()

Sha                           694
NumberOfChatgptSharings       694
Status404                       2
ModelGPT4                     606
ModelGPT3.5                    50
ModelOther                     36
TotalNumberOfPrompts         1922
TotalTokensOfPrompts       585480
TotalTokensOfAnswers       666312
NumberOfConversations        1602
dtype: int64

In [36]:
df_repo.describe()

,Sha,NumberOfChatgptSharings,Status404,ModelGPT4,ModelGPT3.5,ModelOther,TotalNumberOfPrompts,TotalTokensOfPrompts,TotalTokensOfAnswers,NumberOfConversations
count,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000
mean,9.131579,9.131579,0.026316,7.973684,0.657895,0.473684,25.289474,7703.684211,8767.263158,21.078947
std,65.818405,65.818405,0.161136,63.857753,0.664382,2.003155,110.871074,48888.175826,40485.195534,85.291150
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,2.000000,150.500000,530.750000,1.000000
50%,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,6.000000,788.000000,2029.500000,4.000000
75%,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,17.750000,1692.500000,4929.750000,16.500000
max,575.000000,575.000000,1.000000,557.000000,3.000000,17.000000,967.000000,426489.000000,352976.000000,742.000000


# Per-repository info

In [42]:
repo_df = df = pd.read_csv('../data/interim/commit_sharings_groupby_repo_df.csv', index_col='RepoName')
repo_df

,RepoLanguage,Sha,NumberOfChatgptSharings,Status404,ModelGPT4,ModelGPT3.5,ModelOther,TotalNumberOfPrompts,TotalTokensOfPrompts,TotalTokensOfAnswers,NumberOfConversations,is_cloned
RepoName,,,,,,,,,,,,
4lejandrito/fetchbook,JavaScript,1,1,0,0,1,0,5,328,2543,4,True
Atulaya123/reddit-next13-prisma-sql,CSS,1,1,0,0,1,0,2,189,209,1,True
Buildstarted/linksfordevs,CSS,1,1,0,0,1,0,64,516,7473,63,True
CPSC-1520/CPSC-1520-Workbook-A02-dgilleland,CSS,1,1,0,0,0,1,1,267,605,1,True
Cissucalris9/cuemaker2,Python,1,1,0,0,1,0,17,5299,7520,16,True
...,...,...,...,...,...,...,...,...,...,...,...,...
veevyo/coding-exercises,C,1,1,0,0,1,0,1,10,533,1,True
waylon620/GPT-tutor,Batchfile,1,1,0,0,1,0,3,69,923,2,True
winksaville/scripts,Python,1,1,0,1,0,0,27,1441,7331,26,True


In [43]:
repo_df.describe()

,Sha,NumberOfChatgptSharings,Status404,ModelGPT4,ModelGPT3.5,ModelOther,TotalNumberOfPrompts,TotalTokensOfPrompts,TotalTokensOfAnswers,NumberOfConversations
count,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000
mean,9.131579,9.131579,0.026316,7.973684,0.657895,0.473684,25.289474,7703.684211,8767.263158,21.078947
std,65.818405,65.818405,0.161136,63.857753,0.664382,2.003155,110.871074,48888.175826,40485.195534,85.291150
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,2.000000,150.500000,530.750000,1.000000
50%,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,6.000000,788.000000,2029.500000,4.000000
75%,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,17.750000,1692.500000,4929.750000,16.500000
max,575.000000,575.000000,1.000000,557.000000,3.000000,17.000000,967.000000,426489.000000,352976.000000,742.000000


In [47]:
repo_df[['RepoLanguage', 'is_cloned']].nunique()

RepoLanguage    18
is_cloned        1
dtype: int64

In [50]:
repo_df['RepoLanguage'].value_counts()

RepoLanguage
CSS                 21
Python              10
JavaScript           6
Dockerfile           5
Shell                5
C                    3
Batchfile            3
HTML                 3
Cython               2
Lua                  2
EJS                  1
C#                   1
Jinja                1
GAML                 1
Pascal               1
Go                   1
Jupyter Notebook     1
Java                 1
Name: count, dtype: int64

In [51]:
repo_df['is_cloned'].value_counts()

is_cloned
True    76
Name: count, dtype: int64